# Clean code to crop and concatenate drone images 

Tools to crop big drone images into  smaller images that can be used with Deeplabv3+ model to make prediction, and also codes that take the prediction for the croped images and concatenate them to get back to the original size. There is also a function that superimpose the predicted mask on top of the drone images.



In [1]:
from image_slicer import save_tiles, slice
from pathlib import Path
from PIL import Image
import numpy as np
from math import sqrt, ceil, floor
import os
import glob


def get_concat_h(im2, im1):
    dst = Image.new('RGB', (im2.width + im1.width, im2.height))
    dst.paste(im2, (0, 0))
    dst.paste(im1, (im2.width, 0))
    return dst

def get_concat_v(im2, im1):
    dst = Image.new('RGB', (im2.width, im2.height + im1.height))
    dst.paste(im2, (0, 0))
    dst.paste(im1, (0, im2.height))
    return dst

def concat_masks(mask_path, croped_masks_path,images_path,num_cols,num_rows,new_height,new_width,mask_type="png",image_type="jpg"):
    os.makedirs(mask_path, exist_ok=True)
    croped_filenames = glob.glob(os.path.join(croped_masks_path,"*"+mask_type))
    files_names = glob.glob(os.path.join(images_path,"*"+image_type))
    images_names = [Path(x).stem for x in files_names]
    for name in images_names:
        #print(name)
        croped_mask_path = os.path.join(croped_masks_path, Path(name).stem + "_{row:02d}_{col:02d}." + mask_type)
        row_images = []
        for i in range(num_rows):
            path_i_1 = croped_mask_path.format(row = i+1, col = 1)
            im_i_1 = Image.open(path_i_1)
            for j in range(1,num_cols):
                path_i_j = croped_mask_path.format(row = i+1, col = j+1)
                im_i_j = Image.open(path_i_j)
                im_i_1 = get_concat_h(im_i_1, im_i_j)      
            row_images.append(im_i_1)
        im_1 = row_images[0]
        for i in range(1,num_rows):
            im_1 = get_concat_v(im_1,row_images[i])
            
        concat_mask_path = os.path.join(mask_path, name + "." + mask_type)
        #print(concat_mask_path)
        im_1 = im_1.resize((new_height,new_width))
        im_1.save(concat_mask_path)
        

def crop_images(images_path, croped_images_path,crop_num,image_type):
    os.makedirs(croped_images_path, exist_ok=True)
    files_names = glob.glob(os.path.join(images_path,"*"+image_type))
    for file_name in files_names:
        croped_images = slice(file_name, crop_num, save=False)
        save_tiles(croped_images, prefix=Path(file_name).stem, directory=croped_images_path, format="JPEG")
               
def resize_images(croped_images_path,new_height,new_width,image_type):
    files_names = glob.glob(os.path.join(croped_images_path,"*"+image_type))
    for filename in files_names:
        im = Image.open(filename)
        im = im.resize((new_height,new_width))
        im.save(filename)
        
        
def superimpose_images_masks(superimposed_path,images_path,masks_path):
    os.makedirs(superimposed_path, exist_ok=True)
    files_names = glob.glob(os.path.join(images_path,"*"+"JPG"))
    images_names = [Path(x).stem for x in files_names]
    for name in images_names:
        mask_file_path = os.path.join(masks_path, name + "." + "png")
        image_file_path = os.path.join(images_path, name + "." + "JPG")
        superimposed_file_path = os.path.join(superimposed_path, name + "." + "jpg")
        image = Image.open(image_file_path)
        background = image.convert('RGBA')
        mask = Image.open(mask_file_path)
        foreground = mask.convert('RGBA')
        superimp =Image.blend(background, foreground, alpha=.35)
        superimp = superimp.convert('RGB')
        superimp.save(superimposed_file_path)
        

In [188]:
images_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/images/"
croped_images_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/images_croped_2/"
crop_images(images_path, croped_images_path,crop_num=12,image_type="JPG")

In [200]:
resize_images(croped_images_path,new_height=512,new_width=512,image_type="jpg")

In [225]:
images_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/images/"
croped_masks_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/masks_croped_2/"
masks_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/masks_2/"
#concat_masks(masks_path, croped_masks_path,images_path,num_cols=4,num_rows=3,new_height=4000,new_width=3000,mask_type="png",image_type="JPG")

In [226]:
superimposed_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/superimposed_2/"
superimpose_images_masks(superimposed_path,images_path,masks_path)

In [2]:
import numpy as np
filename = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/superimposed_2/DJI_0113.JPG"
im_org = Image.open(filename)

In [6]:
data = np.asarray(im_org)

In [7]:
data.shape

(3000, 4000, 3)

# Diecece 32 

In [223]:
images_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/images/"
croped_masks_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/masks_croped_dicece32/"
masks_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/masks_dicece32/"
concat_masks(masks_path, croped_masks_path,images_path,num_cols=4,num_rows=3,new_height=4000,new_width=3000,mask_type="png",image_type="JPG")

In [224]:
superimposed_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/superimposed_dicece32/"
superimpose_images_masks(superimposed_path,images_path,masks_path)

# Dicece32 probs

In [227]:
images_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/images/"
croped_masks_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/masks_croped_probs_dicece32/"
masks_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/masks_probs_dicece32/"
concat_masks(masks_path, croped_masks_path,images_path,num_cols=4,num_rows=3,new_height=4000,new_width=3000,mask_type="png",image_type="JPG")

In [228]:
superimposed_path = "/Users/stouzani/Desktop/Unstructured_ML/Drone/Drone_Data_Capture/pix4d_HQ/dji_demo/superimposed_dicece32_probs/"
superimpose_images_masks(superimposed_path,images_path,masks_path)